Imported Libaries

In [1]:
import os
root_dir = r'c:\Users\jf79\Cloned Repos\Footfall-Repo'
os.chdir(root_dir)

import pandas as pd

import myfuncs.myfuncs as mf
import myfuncs.footfallfuncs as ff

mf.read_directory()

import warnings
warnings.filterwarnings('ignore')

Your Current Directory is: c:\Users\jf79\Cloned Repos\Footfall-Repo
Files in: ['.git', 'Footfall.venv', 'Footfall_Scripts', 'myfuncs', 'README.md']



In [2]:
cwd = r'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data'
mf.read_directory(cwd)

Directory being read is: C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data
Files in: ['Area Based', 'Hex Based', 'LSOA Based', 'Mapping']



In [3]:
footfall_2024_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/hex_3hourly_counts_2024.csv')
hex_to_borough_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/BT Hex ID to Borough Lookup Table.csv')
relevant_hexes_data = pd.read_csv(f'{cwd}/Hex Based/Relevant Hexes/Relevant Hexes.csv')
bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')

In [ ]:
values_to_keep = [
    "Sands End","Fulham Town","Parsons Green & Sandford","Palace & Hurlingham","Munster",
    "Walham Green","Lillie","Fulham Reach","West Kensington","Hammersmith Broadway",
    "Avonmore","Ravenscourt","Brook Green","Grove","Addison","Wendell Park",
    "Coningham","Shepherd's Bush Green","Wormholt","White City","College Park & Old Oak"
]

bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')
ward_footfall = bespoke_areas_data.copy()

ward_footfall = ward_footfall[ward_footfall['name'].isin(values_to_keep)]
ward_footfall['hours'] = ward_footfall['hours'].str.strip("'")
ward_footfall.rename(
    columns={
        'hours':'time_indicator',
    },
    inplace=True
)
ward_footfall


,bespoke_area_id,name,count_date,time_indicator,resident,visitor,worker,ave_loyalty_percentage,ave_dwell_time
1953166,257,Sands End,2022-05-01,00-03,957,179,89,20.26,109.56
1953167,257,Sands End,2022-05-01,03-06,12538,4633,969,2.16,104.36
1953168,257,Sands End,2022-05-01,06-09,13383,5149,1224,3.42,139.48
1953169,257,Sands End,2022-05-01,09-12,15997,8379,1660,6.97,100.04
1953170,257,Sands End,2022-05-01,12-15,14451,10695,1972,5.95,88.25
...,...,...,...,...,...,...,...,...,...
2122626,277,College Park & Old Oak,2025-02-09,09-12,31155,13968,5345,7.69,87.92
2122627,277,College Park & Old Oak,2025-02-09,12-15,30906,21957,5956,9.36,73.75
2122628,277,College Park & Old Oak,2025-02-09,15-18,30295,23090,6002,8.35,74.16
2122629,277,College Park & Old Oak,2025-02-09,18-21,31449,17293,4932,6.38,83.96


In [ ]:
lon_footfall_2024 = footfall_2024_data.copy()
relevant_hexes = relevant_hexes_data.copy()
hex_to_borough = hex_to_borough_data.copy()

borough_footfall_2024 = pd.merge(
    lon_footfall_2024,
    hex_to_borough,
    right_on='Hex_ID',
    left_on='hex_id',
    how='left'
)

borough_footfall_2024 = borough_footfall_2024.groupby(['borough_name','count_date','time_indicator']).agg(
    resident = ('resident','sum'),
    worker = ('worker','sum'),
    visitor = ('visitor','sum')
).reset_index()

hf_footfall_2024 = pd.merge(
    relevant_hexes_data,
    lon_footfall_2024,
    left_on='Hex_ID',
    right_on='hex_id',
    how='left'
)

In [20]:
typical_day_averages = ff.typical_footfall(
    borough_footfall_2024, '2024-01-01', '2024-12-31',
    footfall_type=['residents','workers','visitors'],
    day_night=False,
    primary_key='borough_name'
)

for i in range(len(typical_day_averages)):
    if 'averages' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='averages', ascending=False))
    elif 'daytime_mean' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='daytime_mean', ascending=False))
    else:
        display(typical_day_averages[i])

Calculating typical daily footfall...
For Weedays and Weekends and Weekly averages...


Aggregating footfall data...
Missing kwargs: {'day_night'}
These args will be set to default values

Applying features...
Features applied.


Detecting anomalies...
613 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
0 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
285 anomalies have been detected.
Anomalies have been flagged and corrected.

Footfall Data Aggregated.



,year,borough_name,averages
32,2024,Westminster,927349.593286
5,2024,Camden,508017.449454
29,2024,Tower Hamlets,420934.896956
27,2024,Southwark,397351.518540
21,2024,Lambeth,373380.353435
24,2024,Newham,351479.153981
31,2024,Wandsworth,327343.258587
1,2024,Barnet,293532.972678
3,2024,Brent,288555.349727
8,2024,Ealing,278621.313817


,year,week_name,borough_name,averages
32,2024,Weekday,Westminster,917723.528626
5,2024,Weekday,Camden,514992.375954
29,2024,Weekday,Tower Hamlets,418187.548800
27,2024,Weekday,Southwark,391263.018539
21,2024,Weekday,Lambeth,360744.654853
24,2024,Weekday,Newham,339145.201745
31,2024,Weekday,Wandsworth,315410.299618
1,2024,Weekday,Barnet,296926.440840
16,2024,Weekday,Hillingdon,281069.980916
6,2024,Weekday,City of London,278768.461832


,year,week_name,borough_name,averages
65,2024,Weekend,Westminster,951599.871566
38,2024,Weekend,Camden,490446.000000
62,2024,Weekend,Tower Hamlets,427856.100962
60,2024,Weekend,Southwark,412689.855082
54,2024,Weekend,Lambeth,405212.594093
57,2024,Weekend,Newham,382551.225962
64,2024,Weekend,Wandsworth,357405.135989
36,2024,Weekend,Brent,318715.344780
44,2024,Weekend,Hackney,295366.659341
41,2024,Weekend,Ealing,289714.155907


In [21]:
mf.export_to_csv(typical_day_averages[0])

Exproting typical_borough_averages  to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\typical_borough_averages .csv

Successfully exported typical_borough_averages  to CSV
